In [1]:
import numpy as np

### Parâmetros

In [56]:
d = 30
tamanho_pop = 50
qte_filhos = 350
max_geracoes = 100
taxa_mut = 0.1

### Funções de fitness

In [39]:
def ackley(x):
    a = 20
    b = 0.2
    c = 2 * np.pi
    term1 = -a * np.exp(-b * np.sqrt(np.sum(x**2) / d))
    term2 = -np.exp(np.sum(np.cos(c * x)) / d)
    return term1 + term2 + a + np.exp(1)

In [40]:
def rastrigin(x):
    return 10 * d + np.sum(x**2 - 10 * np.cos(2 * np.pi * x))

In [41]:
def schwefel(x):
    return 418.9829 * d - np.sum(x * np.sin(np.sqrt(np.abs(x))))

In [42]:
def rosenbrock(x):
    return np.sum(100 * (x[1:] - x[:-1]**2)**2 + (1 - x[:-1])**2)

### Inicialização da população
A população é representada por uma matriz bidimensional, na qual cada linha representa um indivíduo e cada coluna representa um gene. Dependendo da função que queremos analisar, definimos um intervalo de valores diferentes para os genes.

In [43]:
def init_populacao(f):
    if f == ackley:
      return np.random.uniform(-32.768, 32.768, size=(tamanho_pop, d))
    elif f == rastrigin:
      return np.random.uniform(-5.12, 5.12, size=(tamanho_pop, d))
    elif f == schwefel:
      return np.random.uniform(-500, 500, size=(tamanho_pop, d))
    elif f == rosenbrock:
      return np.random.uniform(-2.048, 2.048, size=(tamanho_pop, d))

### Seleção de pais
Torneio de 5 indivíduos aleatórios, selecionando o melhor.

In [58]:
def torneio(populacao, fitness_populacao, tam_torneio):
    ind_torneio = np.random.choice(len(populacao), size=tam_torneio, replace=False)
    fitness_torneio = fitness_populacao[ind_torneio]
    ind_melhor = ind_torneio[np.argmin(fitness_torneio)]
    return populacao[ind_melhor]

### Recombinação intermediária
Define uma combinação linear entre os cromossomos dos dois pais. Alpha é uma matriz com valores aleatórios entre 0 e 1.

In [45]:
def crossover(pai1, pai2):
    alpha = np.random.rand(d)
    filho = alpha * pai1 + (1 - alpha) * pai2
    return filho

### Mutação Gaussiana
Índices aleatórios dos genes do filho gerado são escolhidos para ser aplicada a mutação, com base na taxa definida. A partir daí, são adicionados nesses genes valores de uma distribuição gaussiana com média 0 e desvio padrão 1.

In [46]:
def mutacao(filho):
    mascara_mut = np.random.rand(d) < taxa_mut
    filho[mascara_mut] += np.random.normal(0, 1, np.sum(mascara_mut))
    return filho

### Seleção de sobreviventes
É usada a estratégia ( μ , λ ) de seleção de sobreviventes, na qual λ = 7 . μ, sendo selecionados os 50 melhores filhos entre os 350.

In [60]:
def selecao_sobreviventes(populacao_nova, funcao_fitness):
    sobreviventes = sorted(populacao_nova, key=funcao_fitness)
    sobreviventes = sobreviventes[:50]
    return sobreviventes

### Algoritmo de Estratégia Evolutiva

In [61]:
def estrategia_evolutiva(funcao_fitness):
    populacao = init_populacao(funcao_fitness)

    for geracao in range(max_geracoes):
        fitness_populacao = np.array([funcao_fitness(individuo) for individuo in populacao])

        melhor_indice = np.argmin(fitness_populacao)
        melhor_individuo = populacao[melhor_indice]
        melhor_fitness = fitness_populacao[melhor_indice]

        if geracao % 10 == 0:
            print(f"Geração {geracao}: Melhor Fitness = {melhor_fitness}")

        populacao_nova = []

        while len(populacao_nova) < qte_filhos:
            pai1 = torneio(populacao, fitness_populacao, tam_torneio=5)
            pai2 = torneio(populacao, fitness_populacao, tam_torneio=5)

            filho = crossover(pai1, pai2)
            child = mutacao(filho)

            populacao_nova.append(filho)

        sobreviventes = selecao_sobreviventes(populacao_nova, funcao_fitness)
        populacao = np.array(sobreviventes)

    return melhor_individuo, melhor_fitness

### Rodando a EE para a função Ackley

In [62]:
melhor_solucao, melhor_fitness = estrategia_evolutiva(ackley)
print(f"Melhor Solução: {melhor_solucao}")
print(f"Melhor Fitness: {melhor_fitness}")

Geração 0: Melhor Fitness = 21.021836313692003
Geração 10: Melhor Fitness = 7.260796033710825
Geração 20: Melhor Fitness = 3.7049031286059813
Geração 30: Melhor Fitness = 1.534838920013311
Geração 40: Melhor Fitness = 0.4504263553857588
Geração 50: Melhor Fitness = 0.3128261028290926
Geração 60: Melhor Fitness = 0.25336184157970054
Geração 70: Melhor Fitness = 0.21122013751543678
Geração 80: Melhor Fitness = 0.16399489277278478
Geração 90: Melhor Fitness = 0.1091163840076459
Melhor Solução: [-0.00984718  0.00621031 -0.00458832  0.01614136  0.01930142 -0.00823515
 -0.01731824  0.00950004  0.03221017 -0.00452735 -0.00908366  0.00568091
  0.0138306   0.02088296 -0.01889586  0.01131234  0.00683289 -0.01359861
  0.02776124  0.00535955 -0.01098643 -0.0102693  -0.00155372 -0.02217831
 -0.04988917  0.0043977   0.03389231  0.00544066 -0.02388791 -0.00719977]
Melhor Fitness: 0.08844449872927074


### Rodando a EE para a função Rastrigin

In [63]:
melhor_solucao, melhor_fitness = estrategia_evolutiva(rastrigin)
print(f"Melhor Solução: {melhor_solucao}")
print(f"Melhor Fitness: {melhor_fitness}")

Geração 0: Melhor Fitness = 424.93922995891603
Geração 10: Melhor Fitness = 108.52969097687418
Geração 20: Melhor Fitness = 82.1355363238468
Geração 30: Melhor Fitness = 63.128182904408476
Geração 40: Melhor Fitness = 49.11746102163826
Geração 50: Melhor Fitness = 47.052332260538066
Geração 60: Melhor Fitness = 39.309863113255346
Geração 70: Melhor Fitness = 30.142863894569473
Geração 80: Melhor Fitness = 33.32267448862916
Geração 90: Melhor Fitness = 42.89836970646195
Melhor Solução: [ 0.03516815  1.01651988  1.00856613  0.02176771  0.00353558  0.03494691
  0.88660838  1.05034438 -0.02523627  0.03975763  1.95678395  1.03668581
 -0.13356552 -0.01947289 -0.92303295  0.0412891   1.0205679  -1.01260083
 -0.01214501  0.12417111  1.01220783  0.95464279  0.02135465  0.81471915
  0.03106602  1.05042123 -0.99436253  0.05092459 -0.07623259 -0.93574281]
Melhor Fitness: 39.63283499725395


### Rodando a EE para a função Schwefel

In [64]:
melhor_solucao, melhor_fitness = estrategia_evolutiva(schwefel)
print(f"Melhor Solução: {melhor_solucao}")
print(f"Melhor Fitness: {melhor_fitness}")

Geração 0: Melhor Fitness = 10047.838137651624
Geração 10: Melhor Fitness = 6887.704696673001
Geração 20: Melhor Fitness = 6694.476822082645
Geração 30: Melhor Fitness = 6485.365727761715
Geração 40: Melhor Fitness = 6310.737063001298
Geração 50: Melhor Fitness = 6107.865770089749
Geração 60: Melhor Fitness = 5956.272912702068
Geração 70: Melhor Fitness = 5804.109192946136
Geração 80: Melhor Fitness = 5645.393957031505
Geração 90: Melhor Fitness = 5534.61573422064
Melhor Solução: [ -26.02958233 -118.9154948   196.55189204 -293.16647467  422.49328063
 -129.14200779 -301.74496153  202.77770444 -300.83347565 -301.1119992
  414.09428434  413.82933882  201.30735178 -308.20727274 -296.11288301
 -302.95250058 -297.50338065 -131.88337859  201.36755916  192.4744437
 -125.10055231  198.1495506    11.1999391  -120.41030958 -121.43271898
  419.53066575  192.49957072  425.3797311  -123.71903343  420.29091534]
Melhor Fitness: 5457.708904133631


### Rodando a EE para a função Rosebrock

In [65]:
melhor_solucao, melhor_fitness = estrategia_evolutiva(rosenbrock)
print(f"Melhor Solução: {melhor_solucao}")
print(f"Melhor Fitness: {melhor_fitness}")

Geração 0: Melhor Fitness = 8571.733313399794
Geração 10: Melhor Fitness = 73.24509282130154
Geração 20: Melhor Fitness = 42.89013521238823
Geração 30: Melhor Fitness = 36.87828244125832
Geração 40: Melhor Fitness = 34.08646625484983
Geração 50: Melhor Fitness = 31.682448988520285
Geração 60: Melhor Fitness = 30.556835336351053
Geração 70: Melhor Fitness = 29.924529353012
Geração 80: Melhor Fitness = 29.26713207687928
Geração 90: Melhor Fitness = 28.94024181733437
Melhor Solução: [ 0.19036252  0.02413241  0.0083825   0.01961787  0.02154182  0.02237889
  0.02270819  0.04375988  0.02868429  0.02466531  0.03699274  0.00855914
  0.01367187  0.01683902  0.00541586  0.02076243  0.02876327  0.02375099
  0.01717242 -0.00489709  0.01703579  0.01951606  0.03057921  0.02658862
  0.00293713  0.00569693  0.01456282  0.02081196  0.02039659  0.0080807 ]
Melhor Fitness: 28.831800576807197
